# Boyer-Moore String Matching Practical

This notebook demonstrates the implementation and application of the Boyer-Moore string matching algorithm, including the bad character rule, good suffix rule, and match skip functionality.

In [ ]:
# Define preprocessing utilities
def z_array(s):
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s) - 1)
    for i in range(1, len(s)):
        if s[i] == s[i - 1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        if k > r:
            for i in range(k, len(s)):
                if s[i] == s[i - k]:
                    z[k] += 1
                else:
                    break
            if z[k] > 0:
                r, l = k + z[k] - 1, k
        else:
            beta_len = r - k + 1
            z_kp = z[k - l]
            if z_kp < beta_len:
                z[k] = z_kp
            else:
                match_len = 0
                for i in range(r + 1, len(s)):
                    if s[i] == s[i - k]:
                        match_len += 1
                    else:
                        break
                z[k] = r - k + 1 + match_len
                r, l = k + z[k] - 1, k
    return z

def n_array(s):
    return z_array(s[::-1])[::-1]

def big_l_prime_array(p, n):
    lp = [0] * len(p)
    for j in range(len(p) - 1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp

def small_l_prime_array(n):
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i + 1:
            small_lp[len(n) - i - 1] = i + 1
    for i in range(len(n) - 2, -1, -1):
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i + 1]
    return small_lp

def good_suffix_table(p):
    n = n_array(p)
    big_l_prime = big_l_prime_array(p, n)
    small_l_prime = small_l_prime_array(n)
    return big_l_prime, small_l_prime

def dense_bad_char_tab(p, amap):
    tab = []
    nxt = [0] * len(amap)
    for i in range(len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i + 1
    return tab

class BoyerMoore:
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        self.amap = {c: i for i, c in enumerate(alphabet)}
        self.bad_char = dense_bad_char_tab(p, self.amap)
        self.big_l_prime, self.small_l_prime = good_suffix_table(p)

    def bad_character_rule(self, i, c):
        assert c in self.amap
        ci = self.amap[c]
        return i - (self.bad_char[i][ci] - 1)

    def good_suffix_rule(self, i):
        length = len(self.big_l_prime)
        assert i < length
        if i == length - 1:
            return 0
        i += 1
        if self.big_l_prime[i] > 0:
            return length - self.big_l_prime[i]
        return length - self.small_l_prime[i]

    def match_skip(self):
        return len(self.small_l_prime) - self.small_l_prime[0]

In [ ]:
# Boyer-Moore search function
def boyer_moore(p, p_bm, t):
    """Boyer-Moore string matching algorithm."""
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p) - 1, -1, -1):
            if p[j] != t[i + j]:
                skip_bc = p_bm.bad_character_rule(j, t[i + j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [ ]:
# Test the implementation
text = "ACACGCTC"
pattern = "ACAC"
p_bm = BoyerMoore(pattern)
matches = boyer_moore(pattern, p_bm, text)
print("Pattern found at positions:", matches)

In [ ]:
# Small Exercise: Test Boyer-Moore on a sample text and pattern
# Text: contains two occurrences of the pattern 'TCTA'
t = 'GCTACGATCTAGAATCTA'
p = 'TCTA'

# Create Boyer-Moore object
p_bm = BoyerMoore(p)

# Run the search
matches = boyer_moore(p, p_bm, t)

# Display results
print("Pattern found at positions:", matches)

# Explanation:
# The pattern 'TCTA' appears at index 7 (TCTAGA...) and again at index 14 (...TCTA)
# These are exact matches identified efficiently using the Boyer-Moore algorithm.

In [ ]:
### Disclaimer and Attribution
'''This repository is not an official course resource. All credit for the original course material goes to the instructors and creators of Algorithms for DNA Sequencing, offered via Coursera and developed by the Johns Hopkins University.

This project is intended solely for non-commercial, documentation and educational purposes as part of my personal learning journey.'''